In [ ]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 42.5 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 27.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 40.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 30.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_json("hf://datasets/itinerai/restaurants/restaurant_data.json")

In [ ]:
df.iloc[0]['HOURS'][0]

[{'open': 600, 'openHours': '10:00', 'close': 1320, 'closeHours': '22:00'}]

In [ ]:
df.columns

Index(['ID', 'NAME', 'TRIPADVISOR_URL', 'RESTAURANT_URL', 'PICTURE', 'RATING',
       'DESTINATION', 'DESCRIPTION', 'LATITUDE', 'LONGITUDE',
       'GENERAL_LOCATION', 'NUMBER_OF_REVIEWS', 'REVIEW_TAGS', 'CUISINES',
       'DISHES', 'MEAL_TYPES', 'FEATURES', 'HOURS', 'PRICE_LEVEL',
       'PRICE_RANGE'],
      dtype='object')

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# Initialize model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
doc_encoder = SentenceTransformer('all-MiniLM-L6-v2', device=device)
doc_encoder = doc_encoder.half()

def create_documents(df):
    documents = []
    for _, row in df.iterrows():
        doc = f"{row['NAME']} is a {row['CUISINES']} restaurant located in {row['GENERAL_LOCATION']}. "
        doc += f"It features {row['FEATURES']}. " if row['FEATURES'] else ""
        doc += f"A brife desciction includes {row['DESCRIPTION']}. " if row['DESCRIPTION'] else ""
        doc += f"Hours: {row['HOURS'][0]}." if row['HOURS'] else ""
        doc += f"Rating: {row['RATING']}/5."
        documents.append(doc)
    return documents

# Generate document embeddings on GPU
documents = create_documents(df)
doc_embeddings = doc_encoder.encode(documents, convert_to_tensor=True, device=device)


In [ ]:
documents

["The Consulate UWS is a ['French', 'American', 'Bar', 'Pub', 'Vegetarian Friendly', 'Vegan Options'] restaurant located in 519 Columbus Ave, 10024-3402. It features ['Gift Cards Available', 'Reservations', 'Outdoor Seating', 'Seating', 'Serves Alcohol', 'Full Bar', 'Accepts Credit Cards', 'Table Service', 'Delivery', 'Takeout', 'Street Parking', 'Highchairs Available', 'Wheelchair Accessible', 'Accepts American Express', 'Accepts Mastercard', 'Accepts Visa', 'Free Wifi']. A brife desciction includes To make direct reservation visit our website The Consulate: Your Passport to a Unique Dining Experience New York, NY - October 2019 - Take your tastebuds on an international trip at the consulate, an exotic restaurant and cocktail bar that aims to surprise and inspire patrons with its unique rotating monthly menu that highlights different countries' cultures. Just a 5 min walking from the Museum of National History and Art And a few steps from Central Park. Hours: [{'open': 600, 'openHours

In [ ]:
doc_embeddings

tensor([[ 0.0443, -0.0405, -0.0228,  ...,  0.0090, -0.0894, -0.0197],
        [ 0.0517, -0.0557, -0.0131,  ..., -0.0375, -0.0883, -0.0008],
        [ 0.0389, -0.0911,  0.0377,  ..., -0.0879, -0.0040,  0.0325],
        ...,
        [ 0.0226, -0.0324,  0.0418,  ..., -0.0010, -0.0698, -0.0273],
        [-0.0333, -0.0211,  0.0028,  ..., -0.0015, -0.0510,  0.1192],
        [ 0.0454, -0.0061, -0.0161,  ..., -0.0938, -0.0563, -0.0475]],
       device='cuda:0', dtype=torch.float16)

In [ ]:
import faiss
import numpy as np

# Convert to numpy array for FAISS
embeddings_np = doc_embeddings.cpu().numpy().astype('float32')

# Create FAISS index on CPU
index = faiss.IndexFlatIP(embeddings_np.shape[1])  # IP = Inner Product
index.add(embeddings_np)


In [ ]:
index

<faiss.swigfaiss_avx512.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7f7bb1f08630> >

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Initialize generator on GPU
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
generator = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)

def naive_rag(query, k=3):
    # Encode query
    query_embedding = doc_encoder.encode(query, convert_to_tensor=True, device=device)

    # FAISS search on GPU
    D, I = index.search(query_embedding.cpu().numpy().reshape(1,-1).astype('float32'), k)

    # Generate response
    context = "\n".join([documents[i] for i in I[0]])
    print(f"context: {context}")
    inputs = tokenizer(f"Query: {query}\nContext: {context}", return_tensors="pt", max_length=3000).to(device)
    outputs = generator.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
from sentence_transformers import CrossEncoder

# Initialize cross-encoder on GPU
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

def rerank_rag(query, k=10, top_n=3):
    # Initial retrieval
    query_embedding = doc_encoder.encode(query, convert_to_tensor=True, device=device)
    D, I = index.search(query_embedding.cpu().numpy().reshape(1,-1).astype('float32'), k)

    # Re-ranking with cross-encoder
    pairs = [(query, documents[i]) for i in I[0]]
    scores = reranker.predict(pairs, batch_size=32, convert_to_tensor=True)

    # Sort and select top_n
    ranked_indices = np.argsort(-scores.cpu().numpy())[:top_n]
    final_context = "\n".join([documents[I[0][i]] for i in ranked_indices])

    # Generate response
    inputs = tokenizer(f"Query: {query}\nContext: {final_context}", return_tensors="pt", max_length=3000).to(device)
    outputs = generator.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
def evaluate_pipelines():
    results = []
    for query in queries:
        naive_result = naive_rag(query)
        rerank_result = rerank_rag(query)

        # Manual evaluation metrics
        results.append({
            "query": query,
            "naive": {
                "response": naive_result,
                "relevance": None,  # Manual scoring
                "diversity": None,
                "fluency": None
            },
            "rerank": {
                "response": rerank_result,
                "relevance": None,
                "diversity": None,
                "fluency": None
            }
        })
    return results

In [ ]:
queries = [
    "I want a cozy Italian restaurant near downtown that’s open late."
    "Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews? ",
    "Where can I eat seafood near the beach with a romantic vibe?",
    "Suggest a casual lunch place in the general area of Chinatown.",
    "Are there any highly-rated spots for dessert after 10 PM?",
    "I’m looking for a halal or vegetarian dinner place near the Riverwalk with highchair avaiable.",
    "Recommend a mid-range price Mexican dog friendly restaurant with quick service.",
    "Any hidden gems for coffee or breakfast with quiet seating?",
    "Find a sushi place with scenic views, serves alcohol, and not too expensive.",
    "I want to go somewhere trendy and lively for drinks and appetizers."
]

In [ ]:
results = evaluate_pipelines()
results

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


context: The Loft is a ['Italian', 'Brew Pub', 'Fast Food', 'Mediterranean', 'European', 'Healthy', 'Romana', 'Lazio', 'Central-Italian', 'Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options'] restaurant located in None. It features ['Takeout', 'Reservations', 'Outdoor Seating', 'Seating', 'Street Parking', 'Wheelchair Accessible', 'Serves Alcohol', 'Wine and Beer', 'Digital Payments', 'Free Wifi', 'Accepts Credit Cards', 'Table Service']. A brife desciction includes Very close to te Vatican museum, a typical Roman fast food. You can taste the traditional food of Rome like Porchetta, specialities from Italy like Prosciutto crudo San Daniele, buffalo mozzarella. You can drink draft beer or a glass of best Italian wine in a wonderful place for a small price.. Hours: [{'open': 435, 'openHours': '07:15', 'close': 1350, 'closeHours': '22:30'}].Rating: 5.0/5.
Italian American Club is a ['Italian', 'Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options'] restaurant located in 23

[{'query': 'I want a cozy Italian restaurant near downtown that’s open late.Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews? ',
  'naive': {'response': "['Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian'",
   'relevance': None,
   'diversity': None,
   'fluency': None},
  'rerank': {'response': 'Query: I want a cozy Italian restaurant near downtown that’s open late.Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews?',
   'relevance': None,
   'diversity': None,
   'fluency': None}},
 {'query': 'Where can I eat seafood near the beach with a romantic vibe?',
  'naive': {'response': 'Bungalow By The Sea',
   'relevance': None,
   'diversity': None,
   'fluency': None},
  'rerank': {'

In [ ]:
results

[{'query': 'I want a cozy Italian restaurant near downtown that’s open late.Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews? ',
  'naive': {'response': "['Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian'",
   'relevance': None,
   'diversity': None,
   'fluency': None},
  'rerank': {'response': 'Query: I want a cozy Italian restaurant near downtown that’s open late.Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews?',
   'relevance': None,
   'diversity': None,
   'fluency': None}},
 {'query': 'Where can I eat seafood near the beach with a romantic vibe?',
  'naive': {'response': 'Bungalow By The Sea',
   'relevance': None,
   'diversity': None,
   'fluency': None},
  'rerank': {'

In [ ]:
import json
with open("rag_results.json", "w") as f:
    json.dump(results, f, indent=4)


Annotated Data

In [8]:
results = [
  {
    "query": "I want a cozy Italian restaurant near downtown that’s open late.Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews? ",
    "naive": {
      "response": "['Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian'",
      "relevance": 0,
      "diversity": "It has just give the keyword from the question, which is 'Italian'.",
      "fluency": 0
    },
    "rerank": {
      "response": "Query: I want a cozy Italian restaurant near downtown that’s open late.Any vegan-friendly brunch spots with wheelchair accessible, outdoor seating and good reviews?",
      "relevance": 0,
      "diversity": 0,
      "fluency": "Just replicated the question. It is not a relevant answer."
    }
  },
  {
    "query": "Where can I eat seafood near the beach with a romantic vibe?",
    "naive": {
      "response": "Bungalow By The Sea",
      "relevance": 5,
      "diversity": "It has give on one restaurant option",
      "fluency": None
    },
    "rerank": {
      "response": "['Bungalow By The Sea', 'Bungalow By The Sea', 'Sharkies Seafood Restaurant', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', 'Bungalow By The Sea', ",
      "relevance": 5,
      "diversity": "Given 2 options",
      "fluency": None
    }
  },
  {
    "query": "Suggest a casual lunch place in the general area of Chinatown.",
    "naive": {
      "response": "['Tian - Asian Cuisine Studio', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian - Asian Market Cafe', 'Tian -",
      "relevance": 5,
      "diversity": "It has give on one restaurant option",
      "fluency": None
    },
    "rerank": {
      "response": "The Cafe is a restaurant that serves sushi and is located in Chinatown.",
      "relevance": 0,
      "diversity": "Not the right answer",
      "fluency": "Fluency is very good"
    }
  },
  {
    "query": "Are there any highly-rated spots for dessert after 10 PM?",
    "naive": {
      "response": "Yes",
      "relevance": 5,
      "diversity": 0,
      "fluency": 5
    },
    "rerank": {
      "response": "Monte Cafe",
      "relevance": 5,
      "diversity": "In addition to what is given, we got addl. info",
      "fluency": 5
    }
  },
  {
    "query": "I’m looking for a halal or vegetarian dinner place near the Riverwalk with highchair avaiable.",
    "naive": {
      "response": "['Petra Bistro', 'The Chicago Diner, Logan Square', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'Stanley', 'S",
      "relevance": 5,
      "diversity": "3 options",
      "fluency": None
    },
    "rerank": {
      "response": "['The Heaven Restaurant', 'Erhan Restaurant', 'The Chicago Diner, Lakeview', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant', 'The Heaven Restaurant'",
      "relevance": 5,
      "diversity": "3 options",
      "fluency": None
    }
  },
  {
    "query": "Recommend a mid-range price Mexican dog friendly restaurant with quick service.",
    "naive": {
      "response": "['Torote Restaurant at Valle del Sol', 'Mexican', 'Torote Restaurant at Valle del Sol', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexi",
      "relevance": 5,
      "diversity": None,
      "fluency": None
    },
    "rerank": {
      "response": "['Esquina Mexicana' is a ['Mexican', 'Latin', 'Bar', 'Vegetarian Friendly', 'Dog Friendly'] restaurant located in 1361 Washington Ave, 33139. It features ['Delivery', 'Reservations', 'Outdoor Seating', 'Seating', 'Wheelchair Accessible', 'Serves Alcohol', 'Table Service', 'Street Parking', 'Highchairs Available', 'Free Wifi', 'Accepts Discover', 'Dog Friendly', 'Family style']",
      "relevance": 5,
      "diversity": None,
      "fluency": "Fluent is giving the description of the restaurant in addition to the restaurant"
    }
  },
  {
    "query": "Any hidden gems for coffee or breakfast with quiet seating?",
    "naive": {
      "response": "Snooze Coffee House",
      "relevance": 5,
      "diversity": 5,
      "fluency": None
    },
    "rerank": {
      "response": "Snooze Coffee House",
      "relevance": 5,
      "diversity": None,
      "fluency": None
    }
  },
  {
    "query": "Find a sushi place with scenic views, serves alcohol, and not too expensive.",
    "naive": {
      "response": "['Sushi Bar', 'Sushi Lab', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashiba', 'Sushi Kashi",
      "relevance": 5,
      "diversity": "3 options given",
      "fluency": None
    },
    "rerank": {
      "response": "['Sushi Kashiba', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sushi Gen', 'Sush",
      "relevance": 5,
      "diversity": "only 2 options given",
      "fluency": None
    }
  },
  {
    "query": "I want to go somewhere trendy and lively for drinks and appetizers.",
    "naive": {
      "response": "I want to go somewhere trendy and lively for drinks and appetizers.",
      "relevance": 0,
      "diversity": None,
      "fluency": None
    },
    "rerank": {
      "response": "If you are looking for a trendy and lively place to go for drinks and appetizers, The Corner Lounge & Bar is the place to go.",
      "relevance": 5,
      "diversity": None,
      "fluency": "Very fluent in responding to the question by adding restaurant name with the question"
    }
  }
]

In [9]:
import pandas as pd
flat_data = []
for item in results:
    row = {
        "query": item["query"],
        "naive_response": item["naive"].get("response"),
        "naive_relevance": item["naive"].get("relevance"),
        "naive_diversity": item["naive"].get("diversity"),
        "naive_fluency": item["naive"].get("fluency"),
        "rerank_response": item["rerank"].get("response"),
        "rerank_relevance": item["rerank"].get("relevance"),
        "rerank_diversity": item["rerank"].get("diversity"),
        "rerank_fluency": item["rerank"].get("fluency"),
    }
    flat_data.append(row)

df = pd.DataFrame(flat_data)
df

,query,naive_response,naive_relevance,naive_diversity,naive_fluency,rerank_response,rerank_relevance,rerank_diversity,rerank_fluency
0,I want a cozy Italian restaurant near downtown...,"['Italian', 'Italian', 'Italian', 'Italian', '...",0,It has just give the keyword from the question...,0.0,Query: I want a cozy Italian restaurant near d...,0,0,Just replicated the question. It is not a rele...
1,Where can I eat seafood near the beach with a ...,Bungalow By The Sea,5,It has give on one restaurant option,NaN,"['Bungalow By The Sea', 'Bungalow By The Sea',...",5,Given 2 options,None
2,Suggest a casual lunch place in the general ar...,"['Tian - Asian Cuisine Studio', 'Tian - Asian ...",5,It has give on one restaurant option,NaN,The Cafe is a restaurant that serves sushi and...,0,Not the right answer,Fluency is very good
3,Are there any highly-rated spots for dessert a...,Yes,5,0,5.0,Monte Cafe,5,"In addition to what is given, we got addl. info",5
4,I’m looking for a halal or vegetarian dinner p...,"['Petra Bistro', 'The Chicago Diner, Logan Squ...",5,3 options,NaN,"['The Heaven Restaurant', 'Erhan Restaurant', ...",5,3 options,None
5,Recommend a mid-range price Mexican dog friend...,"['Torote Restaurant at Valle del Sol', 'Mexica...",5,None,NaN,"['Esquina Mexicana' is a ['Mexican', 'Latin', ...",5,None,Fluent is giving the description of the restau...
6,Any hidden gems for coffee or breakfast with q...,Snooze Coffee House,5,5,NaN,Snooze Coffee House,5,None,None
7,"Find a sushi place with scenic views, serves a...","['Sushi Bar', 'Sushi Lab', 'Sushi Kashiba', 'S...",5,3 options given,NaN,"['Sushi Kashiba', 'Sushi Gen', 'Sushi Gen', 'S...",5,only 2 options given,None
8,I want to go somewhere trendy and lively for d...,I want to go somewhere trendy and lively for d...,0,None,NaN,If you are looking for a trendy and lively pla...,5,None,Very fluent in responding to the question by a...


In [ ]:
pip install transformers datasets accelerate peft trl torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 52.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Load Dolly 15K dataset
full_dataset = load_dataset(
    "databricks/databricks-dolly-15k",
    split="train"
)


In [ ]:
# Move model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15001
})

In [ ]:
full_dataset = full_dataset.train_test_split(
    test_size=10,  # Exact 10 samples
    seed=42
)

# Training data
dataset = full_dataset["train"]

# Held-out evaluation set
test_instructions = full_dataset["test"]["instruction"]  # 10 samples
test_contexts = full_dataset["test"]["context"]  # Corresponding contexts


In [ ]:
def format_prompt(example):
    prompt = f"### Instruction:\n{example['instruction']}\n"
    if example['context']:
        prompt += f"### Context:\n{example['context']}\n"
    if example['category']:
        prompt += f"### Category:\n{example['category']}\n"
    prompt += f"### Response:\n{example['response']}"
    return {"text": prompt}

dataset = dataset.map(format_prompt)


Map: 100%|██████████| 15001/15001 [00:00<00:00, 23489.29 examples/s]


In [ ]:
from trl import DataCollatorForCompletionOnlyLM

response_template = "### Response:\n"
collator = DataCollatorForCompletionOnlyLM(
    response_template=tokenizer.encode(response_template, add_special_tokens=False)[2:],  # Skip BOS token
    tokenizer=tokenizer,
    mlm=False
)


In [ ]:
print(dataset[0]['text'])

### Instruction:
What is the third limb of yoga?
### Context:
sanas originated in India. In his Yoga Sutras, Patanjali (c. 2nd to 4th century CE) describes asana practice as the third of the eight limbs (Sanskrit: अष्टाङ्ग, aṣṭāṅga, from अष्ट् aṣṭ, eight, and अङ्ग aṅga, limb) of classical, or raja yoga.
### Category:
summarization
### Response:
The third limb of yoga is know as Asana.  This refers to the physical practice of yoga, or the yoga postures.  By putting the body into difficult positions you achieve several key benefits. The first is you are helping your body heal pain.  If you want to meditate and work on healing your mind, you need to first heal your body.  Otherwise your easily distracted mind will focus on pain in your body versus the pain in your mind.  Secondly, commitment to the Asana will teach you the discipline your mind needs to heal.  When going into uncomfortable situations, bad patterns in the mind often arise.  Stepping into discomfort actually becomes an oppor

In [ ]:
dataset[0]

{'instruction': 'What is the third limb of yoga?',
 'context': 'sanas originated in India. In his Yoga Sutras, Patanjali (c. 2nd to 4th century CE) describes asana practice as the third of the eight limbs (Sanskrit: अष्टाङ्ग, aṣṭāṅga, from अष्ट् aṣṭ, eight, and अङ्ग aṅga, limb) of classical, or raja yoga.',
 'response': 'The third limb of yoga is know as Asana.  This refers to the physical practice of yoga, or the yoga postures.  By putting the body into difficult positions you achieve several key benefits. The first is you are helping your body heal pain.  If you want to meditate and work on healing your mind, you need to first heal your body.  Otherwise your easily distracted mind will focus on pain in your body versus the pain in your mind.  Secondly, commitment to the Asana will teach you the discipline your mind needs to heal.  When going into uncomfortable situations, bad patterns in the mind often arise.  Stepping into discomfort actually becomes an opportunity to study the mind

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    fp16=True,  # Use mixed precision
    logging_steps=10,
    save_strategy="epoch",
    dataloader_pin_memory=True,     # Add this for faster data transfer to GPU
    dataloader_num_workers=4,       # Optimize CPU data loading
    no_cuda=False                   # Ensure CUDA is not disabled
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=collator,
    formatting_func=lambda x: x["text"]
)


Truncating train dataset: 100%|██████████| 15001/15001 [00:00<00:00, 351666.45 examples/s]


In [ ]:
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss
10,3.891500
20,3.122200
30,2.799800
40,2.697900
50,2.819300
60,2.859600
70,3.027800
80,2.634400
90,2.621100
100,2.739100


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=5625, training_loss=2.408566188897027, metrics={'train_runtime': 707.8611, 'train_samples_per_second': 63.576, 'train_steps_per_second': 7.946, 'total_flos': 8611670980389888.0, 'train_loss': 2.408566188897027})

In [ ]:
# Check if model is on GPU
print(f"Model on CUDA: {next(model.parameters()).is_cuda}")

# Monitor GPU usage during training
print(f"GPU Memory before training: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
trainer.train()
print(f"GPU Memory after training: {torch.cuda.memory_allocated() / 1e9:.2f} GB")


Model on CUDA: True
GPU Memory before training: 2.15 GB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss
10,2.517000
20,2.232700
30,2.108000
40,2.157200
50,2.301700
60,2.395400
70,2.569400
80,2.144500
90,2.189600
100,2.257200


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

GPU Memory after training: 2.15 GB


In [ ]:
base_model_name = "EleutherAI/gpt-neo-125M"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name).cuda()


In [ ]:
# Load test instructions
test_instructions = full_dataset["test"]["instruction"]

def generate_response(model, instruction):
    prompt = f"### Instruction:\n{instruction}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_length=512,
        pad_token_id=tokenizer.eos_token_id
    )
    print(f"decoding {instruction}'s output")
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate base model
base_model_responses = [generate_response(base_model, instr) for instr in test_instructions]

# Evaluate fine-tuned model
finetuned_responses = [generate_response(model, instr) for instr in test_instructions]


decoding Who were the children of the legendary Garth Greenhand, the High King of the First Men in the series A Song of Ice and Fire?'s output
decoding Give me a list of basic ingredients for baking cookies's output
decoding Write a funny and whimsical horoscope reading's output
decoding Who was the first person to do spacewalk?'s output
decoding What causes an airplane wing to fly?'s output
decoding Could a nuclear explosion change the Earth's rotation?'s output
decoding In music theory, what is the circle of fifths?'s output
decoding What should I do on a free afternoon in San Francisco?'s output
decoding Extract the dates from the following.'s output
decoding As per the passage whose works did Narendranath study?'s output
decoding Who were the children of the legendary Garth Greenhand, the High King of the First Men in the series A Song of Ice and Fire?'s output
decoding Give me a list of basic ingredients for baking cookies's output
decoding Write a funny and whimsical horoscope re

In [ ]:
!pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 41.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 47.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 11.4 MB/s eta 0:00:00


In [ ]:
test_instructions[1]

'Give me a list of basic ingredients for baking cookies'

In [ ]:
extract_response(base_model_responses[1])

"I'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should know that I'm not\nsure what you're asking.\n\n### Response:\nI'm not sure what you're asking, but I think you should

In [ ]:
extract_response(finetuned_responses[1])

'1. Water\n2. Eggs\n3. Salt\n4. Pepper\n5. Pepperoni\n6. Brown sugar\n7. Pepperoniini\n8. Brown butter\n9. Brown sugar\n10. Brown butter\n11. Brown butter\n12. Brown sugar\n13. Pepperoniini\n14. Brown sugar\n15. Pepperoniini\n16. Brown butter\n17. Brown sugar\n18. Pepperoniini\n19. Brown sugar\n20. Pepperoniini\n21. Brown butter\n22. Brown sugar\n23. Brown butter\n24. Brown sugar\n25. Pepperoniini\n26. Brown sugar\n27. Brown sugar\n28. Pepperoniini\n29. Brown sugar\n30. Pepperoniini\n31. Brown sugar\n32. Brown sugar\n33. Brown sugar\n34. Pepperoniini\n35. Brown sugar\n36. Brown sugar\n37. Pepperoniini\n38. Brown sugar\n39. Brown sugar\n40. Pepperoniini\n41. Brown sugar\n42. Brown sugar\n43. Brown sugar\n44. Pepperoniini\n45. Brown sugar\n46. Brown sugar\n47. Brown sugar\n48. Pepperoniini\n49. Brown sugar\n50. Pepperoniini\n51. Brown sugar\n52. Brown sugar\n53. Brown sugar\n54. Brown sugar\n55. Brown sugar\n56. Brown sugar\n57. Brown sugar\n58. Brown sugar\n59. Brown sugar\n60. Brown su

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def extract_response(text):
    marker = "### Response:\n"
    # Use partition to split at the marker and get the part after it
    _, _, after = text.partition(marker)
    return after.strip()  # Remove leading/trailing whitespace

In [ ]:
import pandas as pd

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    "Instruction": test_instructions,
    "Base Model Response": base_model_responses,
    "Fine-Tuned Model Response": finetuned_responses
})

# Display as markdown table
comparison_df


,Instruction,Base Model Response,Fine-Tuned Model Response
0,Who were the children of the legendary Garth G...,### Instruction:\nWho were the children of the...,### Instruction:\nWho were the children of the...
1,Give me a list of basic ingredients for baking...,### Instruction:\nGive me a list of basic ingr...,### Instruction:\nGive me a list of basic ingr...
2,Write a funny and whimsical horoscope reading,### Instruction:\nWrite a funny and whimsical ...,### Instruction:\nWrite a funny and whimsical ...
3,Who was the first person to do spacewalk?,### Instruction:\nWho was the first person to ...,### Instruction:\nWho was the first person to ...
4,What causes an airplane wing to fly?,### Instruction:\nWhat causes an airplane wing...,### Instruction:\nWhat causes an airplane wing...
5,Could a nuclear explosion change the Earth's r...,### Instruction:\nCould a nuclear explosion ch...,### Instruction:\nCould a nuclear explosion ch...
6,"In music theory, what is the circle of fifths?","### Instruction:\nIn music theory, what is the...","### Instruction:\nIn music theory, what is the..."
7,What should I do on a free afternoon in San Fr...,### Instruction:\nWhat should I do on a free a...,### Instruction:\nWhat should I do on a free a...
8,Extract the dates from the following.,### Instruction:\nExtract the dates from the f...,### Instruction:\nExtract the dates from the f...
9,As per the passage whose works did Narendranat...,### Instruction:\nAs per the passage whose wor...,### Instruction:\nAs per the passage whose wor...


In [ ]:
comparison_df.to_csv('test_results.csv')

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,                  # Rank dimension
    lora_alpha=32,        # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],  # GPT-Neo attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_in"]  # Critical for instruction tuning
)


In [ ]:
model_name

'EleutherAI/gpt-neo-125M'

In [ ]:
import torch


lora_model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

In [ ]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    fp16=True,
    learning_rate=2e-4,
    max_seq_length=512,
    gradient_checkpointing=True,
    logging_steps=10,
    save_strategy="epoch",
    dataloader_pin_memory=True,     # Add this for faster data transfer to GPU
    dataloader_num_workers=4,       # Optimize CPU data loading
    no_cuda=False                   # Ensure CUDA is not disabled
)

trainer = SFTTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    formatting_func=lambda x: x["text"],
    data_collator=collator
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss
10,3.802900
20,3.100400
30,3.117900
40,3.166200
50,3.031300
60,3.073700
70,2.962900
80,2.954300
90,2.941500
100,2.937900


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=2811, training_loss=2.485640931833431, metrics={'train_runtime': 1065.5141, 'train_samples_per_second': 42.236, 'train_steps_per_second': 2.638, 'total_flos': 1.074896298058752e+16, 'train_loss': 2.485640931833431})

In [ ]:
lora_responses = [generate_response(lora_model, instr) for instr in test_instructions]
comparison_df["LoRA Fine-Tuned Response"] = lora_responses


decoding Who were the children of the legendary Garth Greenhand, the High King of the First Men in the series A Song of Ice and Fire?'s output
decoding Give me a list of basic ingredients for baking cookies's output
decoding Write a funny and whimsical horoscope reading's output
decoding Who was the first person to do spacewalk?'s output
decoding What causes an airplane wing to fly?'s output
decoding Could a nuclear explosion change the Earth's rotation?'s output
decoding In music theory, what is the circle of fifths?'s output
decoding What should I do on a free afternoon in San Francisco?'s output
decoding Extract the dates from the following.'s output
decoding As per the passage whose works did Narendranath study?'s output


In [ ]:
lora_responses[1]

'### Instruction:\nGive me a list of basic ingredients for baking cookies\n### Response:\n1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'

In [ ]:
lora_responses

['### Instruction:\nWho were the children of the legendary Garth Greenhand, the High King of the First Men in the series A Song of Ice and Fire?\n### Response:\nTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheTh

In [ ]:
comparison_df

,Instruction,Base Model Response,Fine-Tuned Model Response,LoRA Fine-Tuned Response
0,Who were the children of the legendary Garth G...,### Instruction:\nWho were the children of the...,### Instruction:\nWho were the children of the...,### Instruction:\nWho were the children of the...
1,Give me a list of basic ingredients for baking...,### Instruction:\nGive me a list of basic ingr...,### Instruction:\nGive me a list of basic ingr...,### Instruction:\nGive me a list of basic ingr...
2,Write a funny and whimsical horoscope reading,### Instruction:\nWrite a funny and whimsical ...,### Instruction:\nWrite a funny and whimsical ...,### Instruction:\nWrite a funny and whimsical ...
3,Who was the first person to do spacewalk?,### Instruction:\nWho was the first person to ...,### Instruction:\nWho was the first person to ...,### Instruction:\nWho was the first person to ...
4,What causes an airplane wing to fly?,### Instruction:\nWhat causes an airplane wing...,### Instruction:\nWhat causes an airplane wing...,### Instruction:\nWhat causes an airplane wing...
5,Could a nuclear explosion change the Earth's r...,### Instruction:\nCould a nuclear explosion ch...,### Instruction:\nCould a nuclear explosion ch...,### Instruction:\nCould a nuclear explosion ch...
6,"In music theory, what is the circle of fifths?","### Instruction:\nIn music theory, what is the...","### Instruction:\nIn music theory, what is the...","### Instruction:\nIn music theory, what is the..."
7,What should I do on a free afternoon in San Fr...,### Instruction:\nWhat should I do on a free a...,### Instruction:\nWhat should I do on a free a...,### Instruction:\nWhat should I do on a free a...
8,Extract the dates from the following.,### Instruction:\nExtract the dates from the f...,### Instruction:\nExtract the dates from the f...,### Instruction:\nExtract the dates from the f...
9,As per the passage whose works did Narendranat...,### Instruction:\nAs per the passage whose wor...,### Instruction:\nAs per the passage whose wor...,### Instruction:\nAs per the passage whose wor...


In [ ]:
# Extract only the response
comparison_df['Base Model Extracted Response'] = comparison_df['Base Model Response'].apply(extract_response)
comparison_df['Fine-Tuned Model Extracted Response'] = comparison_df['Fine-Tuned Model Response'].apply(extract_response)
comparison_df['LoRA Fine-Tuned Extracted Response'] = comparison_df['LoRA Fine-Tuned Response'].apply(extract_response)

comparison_df[['Instruction', 'Base Model Extracted Response', 'Fine-Tuned Model Extracted Response', 'LoRA Fine-Tuned Extracted Response']]

,Instruction,Base Model Extracted Response,Fine-Tuned Model Extracted Response,LoRA Fine-Tuned Extracted Response
0,Who were the children of the legendary Garth G...,The children of the Garth Greenhand were the f...,The Garth Greenhand was a famous and influenti...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
1,Give me a list of basic ingredients for baking...,"I'm not sure what you're asking, but I think y...",1. Water\n2. Eggs\n3. Salt\n4. Pepper\n5. Pepp...,1111111111111111111111111111111111111111111111...
2,Write a funny and whimsical horoscope reading,"I'm not sure what the heck I'm doing here, but...",1. I am a child.\n2. I am a child.\n3. I am a ...,1111111111111111111111111111111111111111111111...
3,Who was the first person to do spacewalk?,I think it was the first person to do spacewal...,The first person to do spacewalk was a spacema...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
4,What causes an airplane wing to fly?,The answer is:\n\n* * *\n\n## \n\n## \n\n#...,The wing is the part of the wing that is attac...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
5,Could a nuclear explosion change the Earth's r...,The answer is yes.\n\nA:\n\nThe answer is yes....,The answer is no. The Earth's rotation is caus...,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo...
6,"In music theory, what is the circle of fifths?",I think the answer is:\n\nA:\n\nThe circle of ...,The circle of fifths is the circle of fifths i...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
7,What should I do on a free afternoon in San Fr...,I'm not sure what to do.\n\nA:\n\nYou can do t...,There are many things you can do on a free aft...,ThereThereThereThereThereThereThereThereThereT...
8,Extract the dates from the following.,The response is:\n\n### Example:\n\n```javascr...,The first day of the month is the month of Mar...,••••••••••••••••••••••••••••••••••••••••••••••...
9,As per the passage whose works did Narendranat...,I am not sure if this is correct.\n\nA:\n\nThe...,Rana Patil's The Story of the World is a book ...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...


In [ ]:
comparison_df[['Instruction', 'LoRA Fine-Tuned Extracted Response']]

,Instruction,LoRA Fine-Tuned Extracted Response
0,Who were the children of the legendary Garth G...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
1,Give me a list of basic ingredients for baking...,1111111111111111111111111111111111111111111111...
2,Write a funny and whimsical horoscope reading,1111111111111111111111111111111111111111111111...
3,Who was the first person to do spacewalk?,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
4,What causes an airplane wing to fly?,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
5,Could a nuclear explosion change the Earth's r...,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo...
6,"In music theory, what is the circle of fifths?",TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...
7,What should I do on a free afternoon in San Fr...,ThereThereThereThereThereThereThereThereThereT...
8,Extract the dates from the following.,••••••••••••••••••••••••••••••••••••••••••••••...
9,As per the passage whose works did Narendranat...,TheTheTheTheTheTheTheTheTheTheTheTheTheTheTheT...


In [ ]:
comparison_df.to_csv('fine_tuned_dolly_results.csv')